In [1]:
import pandas as pd
from unused.combine_abc_score import printPanda, gatherPhrases, readScore
from pprint import pprint

In [2]:
phrases = gatherPhrases("../annotations/abc_dataset/tsv/op18_no1_mov1.tsv")
dynamics = pd.read_csv("../annotations/abc_dataset/tables/chords_tables/op18_no1_mov1.tsv", sep="\t")
dynamics = dynamics[dynamics["dynamics"].notna()]
notes = pd.read_csv("../annotations/abc_dataset/tables/notes_tables/op18_no1_mov1.tsv", sep="\t")

FileNotFoundError: [Errno 2] No such file or directory: '../annotations/abc_dataset/tsv/op18_no1_mov1.tsv'

In [3]:
for phrase in phrases:
    printPanda(phrase)

   chord  altchord  measure  beat  totbeat timesig  op  no  mov  length global_key local_key pedal numeral form  figbass changes relativeroot  phraseend
0   .F.I       NaN        1   1.0      1.0     3/4  18   1    1     3.0          F         I   NaN       I  NaN      NaN     NaN          NaN      False
1      V       NaN        2   1.0      4.0     3/4  18   1    1     3.0          F         I   NaN       V  NaN      NaN     NaN          NaN      False
2      I       NaN        3   1.0      7.0     3/4  18   1    1     3.0          F         I   NaN       I  NaN      NaN     NaN          NaN      False
3    IV6       NaN        4   1.0     10.0     3/4  18   1    1     6.0          F         I   NaN      IV  NaN      6.0     NaN          NaN      False
4    V65       NaN        6   1.0     16.0     3/4  18   1    1     3.0          F         I   NaN       V  NaN     65.0     NaN          NaN      False
5      I       NaN        7   1.0     19.0     3/4  18   1    1     1.0          F

In [4]:
printPanda(dynamics)

       mc   mn mc_onset mn_onset timesig  staff  voice duration gracenote nominal_duration scalar  chord_id dynamics articulation  staff_text
1       1    1      1/4      1/4     3/4      1      1        0       NaN              NaN    NaN       NaN        p          NaN         NaN
8       1    1      1/4      1/4     3/4      2      1        0       NaN              NaN    NaN       NaN        p          NaN         NaN
15      1    1      1/4      1/4     3/4      3      1        0       NaN              NaN    NaN       NaN        p          NaN         NaN
22      1    1      1/4      1/4     3/4      4      1        0       NaN              NaN    NaN       NaN        p          NaN         NaN
98      9    9      1/4      1/4     3/4      1      1        0       NaN              NaN    NaN       NaN        f          NaN         NaN
105     9    9      1/4      1/4     3/4      2      1        0       NaN              NaN    NaN       NaN        f          NaN         NaN
112   

In [5]:
printPanda(notes)

       mc   mn mc_onset mn_onset timesig  staff  voice duration     gracenote nominal_duration scalar  tied  tpc  midi  chord_id
0       1    1        0        0     3/4      3      1      1/4           NaN              1/4      1   1.0   -1    53        12
1       1    1        0        0     3/4      4      1      1/4           NaN              1/4      1   1.0   -1    53        18
2       1    1        0        0     3/4      1      1      1/4           NaN              1/4      1   1.0   -1    65         0
3       1    1        0        0     3/4      2      1      1/4           NaN              1/4      1   1.0   -1    65         6
4       1    1      1/4      1/4     3/4      3      1      1/8           NaN              1/8      1  -1.0   -1    53        13
5       1    1      1/4      1/4     3/4      4      1      1/8           NaN              1/8      1  -1.0   -1    53        19
6       1    1      1/4      1/4     3/4      1      1      1/8           NaN              1/8   

In [6]:
results = {i:{"phrase_df":phrase_df} for i, phrase_df in enumerate(phrases)}

In [7]:
for k in results.keys():
    phrase = results[k]["phrase_df"]
    results[k]["harmonic_rhythm_measure_ratio"] = len(phrase)/(phrase["measure"].iloc[-1] - phrase["measure"].iloc[0] + 1)
    # print(results[k])

In [8]:
"""Find the local mode of a phrase. Possibilities are major, minor, and modulating from one to another"""
for k in results.keys():
    phrase = results[k]["phrase_df"]
    # printPanda(phrase)
    same_start_end = phrase["local_key"].iloc[0] == phrase["local_key"].iloc[-1]
    start_is_major = phrase["local_key"].iloc[0][-1].isupper()
    end_is_major = phrase["local_key"].iloc[-1][-1].isupper()
    if same_start_end:
        if start_is_major:
            results[k]["local_mode"] = "major"
        else:
            results[k]["local_mode"] = "minor"
    else:
        if start_is_major and end_is_major:
            results[k]["local_mode"] = "major_to_major"
        elif start_is_major and not end_is_major:
            results[k]["local_mode"] = "major_to_minor"
        elif not start_is_major and end_is_major:
            results[k]["local_mode"] = "minor_to_major"
        else:
            results[k]["local_mode"] = "minor_to_minor"

In [9]:
"""Find all dynamics present in a particular phrase"""
for k in results.keys():
    phrase = results[k]["phrase_df"]
    measures = (phrase["measure"].iloc[0], phrase["measure"].iloc[-1])

    phraseDynamics = dynamics[(measures[0] <= dynamics["mc"]) & (dynamics["mc"] <= measures[1])]
    results[k]["dynamics"] = set(phraseDynamics["dynamics"])

In [10]:
"""Find note ranges present in a particular phrase"""
S_RANGE = tuple(range(73, 102))
A_RANGE = tuple(range(61, 73))
T_RANGE = tuple(range(49, 61))
B_RANGE = tuple(range(22, 49))
for k in results.keys():
    phrase = results[k]["phrase_df"]
    measures = (phrase["measure"].iloc[0], phrase["measure"].iloc[-1])

    measureRanges = notes[(measures[0] <= notes["mc"]) & (notes["mc"] <= measures[1])]

    ranges = {}
    for voiceRange, name in zip([S_RANGE, A_RANGE, T_RANGE, B_RANGE], ["S", "A", "T", "B"]):
        hasRange = any([r in measureRanges["midi"].values for r in voiceRange])
        ranges[name] = hasRange
    results[k]["voice_ranges"] = ranges

In [11]:
for r in results.values():
    pprint(r)
    print()
    print()
    print()

{'dynamics': {'p'},
 'harmonic_rhythm_measure_ratio': 1.25,
 'local_mode': 'major',
 'phrase_df':    chord  altchord  measure  beat  totbeat timesig  op  no  mov  length  \
0   .F.I       NaN        1   1.0      1.0     3/4  18   1    1     3.0   
1      V       NaN        2   1.0      4.0     3/4  18   1    1     3.0   
2      I       NaN        3   1.0      7.0     3/4  18   1    1     3.0   
3    IV6       NaN        4   1.0     10.0     3/4  18   1    1     6.0   
4    V65       NaN        6   1.0     16.0     3/4  18   1    1     3.0   
5      I       NaN        7   1.0     19.0     3/4  18   1    1     1.0   
6     vi       NaN        7   2.0     20.0     3/4  18   1    1     1.0   
7    ii6       NaN        7   3.0     21.0     3/4  18   1    1     1.0   
8  V(64)       NaN        8   1.0     22.0     3/4  18   1    1     2.0   
9  V\\\\       NaN        8   3.0     24.0     3/4  18   1    1     1.0   

  global_key local_key pedal numeral form  figbass changes relativeroot  \
0